In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
data=pd.read_csv('Bengaluru_House_Data.csv')

In [ ]:
data.head()
# data.describe

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data['area_type'].unique()

In [ ]:
data['area_type'].value_counts()

In [ ]:
data2 = data.drop(['area_type','society','balcony','availability'],axis='columns')
data2.shape

In [ ]:

data2.isnull().sum()

In [ ]:
data3 = data2.dropna()
data3.isnull().sum()

In [ ]:

data3['size'].unique()

In [ ]:
data3['bhk'] = data3['size'].apply(lambda x: int(x.split(' ')[0]))
data3.bhk.unique()

In [ ]:

data3.head()

In [ ]:

data3[data3.bhk> 10]

In [ ]:

data3.dtypes

In [ ]:

data3.total_sqft.unique()

In [ ]:
def is_a_float(x):
    try: 
        float(x)
    except:
        return False
    return True

In [ ]:
data3[~data3['total_sqft'].apply(is_a_float)].head(10)

In [ ]:
def takes_maximum(x):
    token = x.split('-') # splitting 2100 - 2850 into two tokens
    if len(token)==2:
        return (max(float(token[0]), float(token[1])))
    try:
        return float(x)
    except:
        return None
        

In [ ]:
takes_maximum('3067 - 8156')

In [ ]:
takes_maximum('3067')

In [ ]:
takes_maximum('34.46Sq. Meter')

In [ ]:
data4 = data3.copy()
data4.total_sqft = data4.total_sqft.apply(takes_maximum)

In [ ]:
data4= data4[data4.total_sqft.notnull()]

In [ ]:
data4.head(10)

In [ ]:
data5 = data4.copy()

In [ ]:
data5['price_per_sqft'] = data5['price']*100000/data5['total_sqft']
data5.head()

In [ ]:
data5.location.unique()

In [ ]:
len(data5.location.unique())

In [ ]:
data5.location = data5.location.apply(lambda x: x.strip())

loc_stat = data5.groupby('location')['location'].agg('count').sort_values(ascending=False)

In [ ]:

loc_stat.unique()

In [ ]:
loc_stats_less_than_10 = loc_stat[loc_stat<=10]

In [ ]:

loc_stats_less_than_10

In [ ]:

len(loc_stats_less_than_10)

In [ ]:
len(data5.location.unique())

In [ ]:
data5.location = data5.location.apply(lambda x: 'other' if x in loc_stats_less_than_10 else x)
len(data5.location.unique())

In [ ]:

data5.head(10)

In [ ]:
data5.groupby('location')['location'].agg('count').sort_values(ascending=True)

In [ ]:
data5.groupby('location')['location'].agg('count').nlargest(n=20).plot(kind='bar',rot=90,figsize=(10, 8),subplots=True,sharex=True,sharey=True,legend=True,grid=True,fontsize=16.0)

In [ ]:

data5[(data5.location=='other')]

In [ ]:
data5[data5.total_sqft/data5.bhk<300].head()

In [ ]:

data6 =data5[~(data5.total_sqft/data5.bhk<300)]

In [ ]:
data6.shape

In [ ]:
data6.price_per_sqft.describe()

In [ ]:
def remove_outlier_ppsft_location(data):
    data_out = pd.DataFrame()
    for key, subdata in data.groupby('location'):
        m = np.mean(subdata.price_per_sqft)
        sd = np.std(subdata.price_per_sqft)
        reduced_data = subdata[(subdata.price_per_sqft > (m - sd)) & (subdata.price_per_sqft <= (m + sd))]
        data_out = pd.concat([data_out, reduced_data], ignore_index=True)
    return data_out

data7 = remove_outlier_ppsft_location(data6)
print(data7.shape)

In [ ]:
def plot_scatter_chart(data, location):
    bhk1 = data[(data.location == location) & (data.bhk == 1)]
    bhk2 = data[(data.location == location) & (data.bhk == 2)]
    bhk3 = data[(data.location == location) & (data.bhk == 3)]
    
    matplotlib.rcParams['figure.figsize'] = (16, 10)
    font = {'family': 'normal',
            'weight': 'bold',
            'size': 22}
    matplotlib.rc('font', **font)
     
    plt.scatter(bhk2.total_sqft, bhk2.price, color='blue', label='2 BHK', s=100)
    plt.scatter(bhk3.total_sqft, bhk3.price, marker='+', color='green', label='3 BHK', s=100)
    plt.scatter(bhk1.total_sqft, bhk1.price, marker='X', color='red', label='1 BHK', s=100)
    
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    
    plt.title(location)
    plt.legend()

plot_scatter_chart(data7, "Whitefield")

In [ ]:
plot_scatter_chart(data7,"Electronic City")

In [ ]:
def remove_bhk_outliers(data):
    exclude_indices = np.array([])
    for location, location_data in data.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_data in location_data.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_data.price_per_sqft),
                'std': np.std(bhk_data.price_per_sqft),
                'count': bhk_data.shape[0]
            }
        for bhk, bhk_data in location_data.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices, bhk_data[bhk_data.price_per_sqft < (stats['mean'])].index.values)
    return data.drop(exclude_indices, axis='index')

data8 = remove_bhk_outliers(data7)


In [ ]:
plot_scatter_chart(data8,"Electronic City")

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
font = {'family' : 'normal',
    'weight' : 'bold',
    'size'   : 22}
matplotlib.rc('font', **font)
plt.hist(data8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
data8.bath.unique()

In [ ]:
data8[(data8['bath']>10)]

In [ ]:
fig, axes = plt.subplots(1, 2)

data8.bath.plot(kind='hist',ax=axes[0],title='Bath')
data8.bhk.plot(kind='hist',ax=axes[1],title='BHK')

plt.show()

In [ ]:
data8[data8.bath> data8.bhk+2]

In [ ]:
data9= data8[data8.bath<data8.bhk+2]
data9.shape

In [ ]:
data9.head(10)

In [ ]:
data10 = data9.drop(['size','price_per_sqft'],axis='columns')

In [ ]:
dummy_col = pd.get_dummies(data10.location)
dummy_col.head(5)

In [ ]:
data11 = pd.concat([data10,dummy_col.drop('other',axis='columns') ], axis='columns' )
data11.head()

In [ ]:
X =data11.drop(['location','price'], axis='columns')

In [ ]:
X.head()

In [ ]:
Y = data11.price
Y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=10)


In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
print ('Linear Regression R score')
print ('----------')
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

In [ ]:
cross_val_score(LinearRegression(), X,Y, cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
    find_best_model_using_gridsearchcv(X,Y)

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [ ]:

predict_price('1st Phase JP Nagar',1000, 2, 2)

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))